<a href="https://colab.research.google.com/github/yuhui-0611/ESAA/blob/main/ESAA_YB_mini_project3_%ED%9A%8C%EA%B7%80%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 한글 폰트 설정
# 나눔 바른 고딕 폰트 설치
!apt-get update -qq
!apt-get install fonts-nanum -qq
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib
import matplotlib.pyplot as plt # Add this line
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
print("한글 폰트 설정 완료")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt # Add this line
plt.rc('font', family='NanumBarunGothic') #matplotlib 한글 깨짐 방지

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 전처리 코드

## 1. train

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('/content/drive/MyDrive/Colab/ESAA/movies_train.csv')

#결측치 처리
train['dir_prev_bfnum'].fillna(0, inplace=True)

#날짜 처리
train['release_time'] = pd.to_datetime(train['release_time'])
train['year'] = train['release_time'].dt.year
train['month'] = train['release_time'].dt.month
train['weekday'] = train['release_time'].dt.day_of_week

#로그 변환
cols_to_transform = ['dir_prev_bfnum', 'num_staff', 'num_actor', 'box_off_num']

for col in cols_to_transform:
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))

    sns.histplot(train[col], ax=ax[0], kde=True)
    ax[0].set_title(f"{col} - Before")

    train[col + "_log"] = np.log1p(train[col])

    sns.histplot(train[col + "_log"], ax=ax[1], kde=True)
    ax[1].set_title(f"{col} - After (log1p)")

    plt.show()

#제거할 원본 변수들
cols_to_drop = [
    'title', 'director', 'release_time',
    'distributor',
    'box_off_num',
    'num_actor', 'num_staff', 'dir_prev_bfnum',
]

train = train.drop(columns=cols_to_drop)

train = train.drop(columns=['dir_prev_num'])

# One-Hot Encoding
cols_to_ohe = ['genre', 'screening_rat', 'year', 'month', 'weekday']
train = pd.get_dummies(train, columns=cols_to_ohe)

#신인 감독 여부 파생 변수
train['is_rookie_director'] = (train['dir_prev_bfnum_log'] == 0).astype(int)

# 결과 확인
print(train.head())
print("train shape:", train.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab/ESAA/movies_train.csv'

## 2. test

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab/ESAA/movies_test.csv')

test['dir_prev_bfnum'].fillna(0, inplace=True)

test['release_time'] = pd.to_datetime(test['release_time'])
test['year'] = test['release_time'].dt.year
test['month'] = test['release_time'].dt.month
test['weekday'] = test['release_time'].dt.day_of_week

cols_to_transform = ['dir_prev_bfnum', 'num_staff', 'num_actor']

for col in cols_to_transform:
    test[col + "_log"] = np.log1p(test[col])

cols_to_drop = [
    'title', 'director', 'release_time',
    'distributor',
    'num_actor', 'num_staff', 'dir_prev_bfnum',
]

test = test.drop(columns=cols_to_drop)

test = test.drop(columns=['dir_prev_num'])

cols_to_ohe = ['genre', 'screening_rat', 'year', 'month', 'weekday']
test = pd.get_dummies(test, columns=cols_to_ohe)

test['is_rookie_director'] = (test['dir_prev_bfnum_log'] == 0).astype(int)

# 회귀 모델

## 1. 성능 평가 방법

In [ ]:
from sklearn.model_selection import train_test_split

y_target=train['box_off_num_log']
X_features=train.drop(columns=['box_off_num_log'],axis=1,inplace=False)
X_train,X_test,y_train,y_test=train_test_split(X_features,y_target,test_size=0.2,random_state=0)

In [ ]:
from sklearn.model_selection import cross_val_score

def get_model_cv_prediction(model,X_data,y_target):
  neg_mse_scores=cross_val_score(model,X_data,y_target,scoring="neg_mean_squared_error",cv=5)
  rmse_scores=np.sqrt(-1*neg_mse_scores)
  avg_rmse=np.mean(rmse_scores)
  print('##### ',model.__class__.__name__,' #####')
  print('5 교차 검증의 평균 RMSE:{0:.3f}'.format(avg_rmse))

In [ ]:
from sklearn.model_selection import GridSearchCV

def print_best_params(model,params):
  grid_model=GridSearchCV(model,param_grid=params,
                          scoring='neg_mean_squared_error',cv=5)
  grid_model.fit(X_train,y_train)
  rmse=np.sqrt(-1*grid_model.best_score_)
  print('{0} 5 CV 시 최적 평균 RMSE 값:{1}, 최적 alpha:{2}'.format(model.__class__.__name__,
                                                           np.round(rmse,4),grid_model.best_params_))

## 2. 회귀 트리 모델 학습/예측/평가

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

dt_reg=DecisionTreeRegressor(random_state=0,max_depth=4)
rf_reg=RandomForestRegressor(random_state=0,n_estimators=1000)
gb_reg=GradientBoostingRegressor(random_state=0,n_estimators=1000)
xgb_reg=XGBRegressor(n_estimators=1000)
lgb_reg=LGBMRegressor(n_estimators=1000)

# 트리 기반의 회귀 모델을 반복하면서 평가 수행
models=[dt_reg,rf_reg,gb_reg,xgb_reg,lgb_reg]
for model in models:
  get_model_cv_prediction(model,X_train,y_train)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [ ]:
get_model_cv_prediction(xgb_reg,X_train,y_train)

#####  XGBRegressor  #####
5 교차 검증의 평균 RMSE:2.049


### 회귀 트리 모델 5 교차 검증의 평균 RMSE

결정트리: 2.096

랜덤포레스트: 1.921

GBM: 2.189

XGB: 2.049

LGBM: 2.028

### 이후 아이디어

각각의 회귀 트리 모델을 하이퍼 파라미터 서치

## 3. 회귀 트리 최적 하이퍼 파라미터 찾기

### 결정트리 하이퍼 파라미터 튜닝

최적 alpha
- max_depth: 3
- min_samples_split: 2

평균 RMSE: 2.096 -> **2.0702**

In [ ]:
from sklearn.model_selection import GridSearchCV

# DecisionTreeRegressor에 맞는 하이퍼파라미터 설정
param_grid={'max_depth': [3, 4, 5, 6, 7],
            'min_samples_split': [2, 5, 10]}
print(param_grid)
# 이전에 정의된 dt_reg 모델 사용
grid_search=GridSearchCV(dt_reg,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,refit=True)

{'max_depth': [3, 4, 5, 6, 7], 'min_samples_split': [2, 5, 10]}


In [ ]:
grid_search.fit(X_train,y_train)
print_best_params(dt_reg,param_grid)

DecisionTreeRegressor 5 CV 시 최적 평균 RMSE 값:2.0702, 최적 alpha:{'max_depth': 3, 'min_samples_split': 2}


### 랜덤포레스트 하이퍼 파라미터 튜닝

최적 alpha
- max_depth: 8
- min_samples_split: 6
- n_estimators: 1000

평균 RMSE: 1.921 -> 1.9213 **커진 것 같진 않고 거의 그대로인 듯!**

In [ ]:
from sklearn.model_selection import GridSearchCV

# RandomForestRegressor에 맞는 하이퍼파라미터 설정
param_grid={
    'n_estimators':[100,200,300,500,1000],
    'max_depth':[8,10,12],
    'min_samples_split':[2,4,6]
}
print(param_grid)
# 이전에 정의된 rf_reg 모델 사용
grid_search=GridSearchCV(rf_reg,param_grid=param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,refit=True)

{'n_estimators': [100, 200, 300, 500, 1000], 'max_depth': [8, 10, 12], 'min_samples_split': [2, 4, 6]}


In [ ]:
grid_search.fit(X_train,y_train)
print_best_params(rf_reg,param_grid)

RandomForestRegressor 5 CV 시 최적 평균 RMSE 값:1.9213, 최적 alpha:{'max_depth': 8, 'min_samples_split': 6, 'n_estimators': 1000}


### GBM 하이퍼 파라미터 튜닝

최적 alpha
- learning_rate: 0.05
- max_depth: 3
- n_estimators: 100

평균 RMSE: 2.189 -> **1.9503**

In [ ]:
from sklearn.model_selection import GridSearchCV

# GBM에 맞는 하이퍼 파라미터 설정
param_grid={
    'n_estimators':[100,200,300,500,1000],
    'learning_rate':[0.05,0.1,0.15],
    'max_depth':[3,5,7]
}
print(param_grid)
# 이전에 정의된 gb_reg 모델 사용
grid_search=GridSearchCV(gb_reg,param_grid=param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,refit=True)

{'n_estimators': [100, 200, 300, 500, 1000], 'learning_rate': [0.05, 0.1, 0.15], 'max_depth': [3, 5, 7]}


In [ ]:
grid_search.fit(X_train,y_train)
print_best_params(gb_reg,param_grid)

GradientBoostingRegressor 5 CV 시 최적 평균 RMSE 값:1.9503, 최적 alpha:{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}


### XGBoost 하이퍼 파라미터 튜닝

최적 alpha
- colsample_bytree: 0.5
- learning_rate: 0.05
- max_depth: 3
- n_estimators: 100
- subsample: 0.5

평균 RMSE: 2.049 -> **1.8677**

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# XGB에 맞는 하이퍼 파라미터 설정
param_grid={
    'n_estimators':[100,200,300,500,1000],
    'learning_rate':[0.05,0.1,0.15],
    'max_depth':[3,5,7],
    'colsample_bytree':[0.5,0.7,0.9],
    'subsample':[0.5,0.7,0.9]
}
print(param_grid)
# 이전에 정의된 xgb_reg 모델 사용
grid_search=GridSearchCV(xgb_reg,param_grid=param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,refit=True)

{'n_estimators': [100, 200, 300, 500, 1000], 'learning_rate': [0.05, 0.1, 0.15], 'max_depth': [3, 5, 7], 'colsample_bytree': [0.5, 0.7, 0.9], 'subsample': [0.5, 0.7, 0.9]}


In [ ]:
grid_search.fit(X_train,y_train)
print_best_params(xgb_reg,param_grid)

XGBRegressor 5 CV 시 최적 평균 RMSE 값:1.8677, 최적 alpha:{'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}


### LightGBM 하이퍼 파라미터 튜닝

최적 alpha
- colsample_bytree: 0.7
- learning_rate: 0.15
- max_depth: 3
- n_estimators: 100
- num_leaves: 32
- subsample: 0.6

평균 RMSE: 2.028 -> **1.8907**

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

# LGBM에 맞는 하이퍼 파라미터 설정
param_grid={
    'n_estimators':[100,200,300,500,1000],
    'learning_rate':[0.05,0.1,0.15],
    'num_leaves':[32,64,128],
    'max_depth':[-1,3,5,7],
    'colsample_bytree':[0.6,0.7,0.8],
    'subsample':[0.6,0.7,0.8]
}
print(param_grid)
# 이전에 정의된 lgb_reg 모델 사용
grid_search=GridSearchCV(lgb_reg,param_grid=param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,refit=True)

{'n_estimators': [100, 200, 300, 500, 1000], 'learning_rate': [0.05, 0.1, 0.15], 'num_leaves': [32, 64, 128], 'max_depth': [-1, 3, 5, 7], 'colsample_bytree': [0.6, 0.7, 0.8], 'subsample': [0.6, 0.7, 0.8]}


In [ ]:
grid_search.fit(X_train,y_train)
print_best_params(lgb_reg,param_grid)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

### 파라미터 튜닝 후 회귀 트리 모델 5 교차 검증의 평균 RMSE

결정트리: 2.096 -> **2.0702**

랜덤포레스트: 1.921 -> **1.9213**

GBM: 2.189 -> **1.9503**

XGB: 2.049 -> **1.8677** -> **1.8652**

LGBM: 2.028 -> **1.8907** -> **1.8823**

### 이후 아이디어

성능이 높은 것 2가지 XGB, LGBM만 하이퍼 파라미터 튜닝 추가로 진행

### XGBoost 추가 하이퍼 파라미터 튜닝

최적 alpha
- colsample_bytree: 0.5
- learning_rate: 0.05
- max_depth: 3
- n_estimators: 100
- subsample: 0.5

추가 alpha
- reg_alpha: 0.01
- reg_lambda: 1

평균 RMSE: 2.049 -> 1.8677 -> **1.8652**

In [ ]:
xgb_reg=XGBRegressor(n_estimators=1000)

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# XGB에 맞는 하이퍼 파라미터 설정
param_grid={
    'n_estimators':[100],
    'learning_rate':[0.05],
    'max_depth':[3],
    'colsample_bytree':[0.5],
    'subsample':[0.5],
    'reg_alpha':[0, 0.01, 0.1, 1],
    'reg_lambda':[0, 0.01, 0.1, 1]
}
print(param_grid)
# 이전에 정의된 xgb_reg 모델 사용
grid_search=GridSearchCV(xgb_reg,param_grid=param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,refit=True)

{'n_estimators': [100], 'learning_rate': [0.05], 'max_depth': [3], 'colsample_bytree': [0.5], 'subsample': [0.5], 'reg_alpha': [0, 0.01, 0.1, 1], 'reg_lambda': [0, 0.01, 0.1, 1]}


### LightGBM 추가 하이퍼 파라미터 튜닝

최적 alpha
- colsample_bytree: 0.7
- learning_rate: 0.15
- max_depth: 3
- n_estimators: 100
- num_leaves: 32
- subsample: 0.6

추가 alpha
- reg_alpha: 0.01
- reg_lambda: 0.1

평균 RMSE: 2.028 -> 1.8907 -> **1.8823**

In [ ]:
from lightgbm import LGBMRegressor
lgb_reg=LGBMRegressor(n_estimators=1000)

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
import lightgbm as lgb

# XGB에 맞는 하이퍼 파라미터 설정
param_grid={
    'n_estimators':[100],
    'learning_rate':[0.05],
    'max_depth':[3],
    'colsample_bytree':[0.5],
    'subsample':[0.5],
    'reg_alpha':[0, 0.01, 0.1, 1, 10],
    'reg_lambda':[0, 0.01, 0.1, 1, 10]
}
print(param_grid)
# 이전에 정의된 xgb_reg 모델 사용
grid_search=GridSearchCV(xgb_reg,param_grid=param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,refit=True)

{'n_estimators': [100], 'learning_rate': [0.05], 'max_depth': [3], 'colsample_bytree': [0.5], 'subsample': [0.5], 'reg_alpha': [0, 0.01, 0.1, 1, 10], 'reg_lambda': [0, 0.01, 0.1, 1, 10]}


In [ ]:
grid_search.fit(X_train,y_train)
print_best_params(lgb_reg,param_grid)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

### 이후 아이디어

XGBoost와 LightGBM의 예측 결과 구해서 합친 뒤 리더보드 점수 확인.

## XGBoost와 LightGBM 혼합 예측

In [ ]:
from sklearn.metrics import mean_squared_error # Add this line

def get_rmse_pred(preds):
  for key in preds.keys():
    pred_value=preds[key]
    mse=mean_squared_error(y_test,pred_value)
    rmse=np.sqrt(mse) # rmsle
    print('{0} 모델의 RMSLE: {1:.3f}'.format(key,rmse))

### 개별 모델 학습과 예측

#### XGBoost 개별 모델

In [ ]:
# xgb
xgb_reg=XGBRegressor(n_estimators=100,
                     colsample_bytree=0.5,
                     learning_rate=0.05,
                     max_depth=3,
                     subsample=0.5,
                     reg_alpha=0.01,
                     reg_lambda=1)
xgb_reg.fit(X_train,y_train)
xgb_pred=xgb_reg.predict(X_test)
print(xgb_pred)

[15.656935  14.284627   7.7857366  7.256275  10.113725   7.6471114
  6.7873173  8.797132   6.8803744  7.6143875  8.0847645 13.405738
  7.248481   6.4382854 14.210645   7.8359375  7.6114926  7.337373
  6.164747   9.234056   8.218965   8.608802  12.907544   6.184027
 15.137897   9.664525  10.171155   7.71119   13.570095   7.851697
 11.283634  13.164945  11.651043   9.21725    8.054158   9.558227
 11.311832   8.07435   11.252398  14.168738   9.74679   10.884388
 15.25611   11.210332   7.929038   8.302114   8.111287   9.908307
  9.719972   7.548364   7.246311  10.384411   9.885733  13.706679
  8.446672   9.356816   7.629811  13.054938   7.5742493  7.419333
  7.13977    8.974814   8.483908   8.290305   7.991209   7.662124
  5.800214  10.7486105  7.0935693  8.919328   7.320208  14.166281
 13.597062  15.299601   7.0508847  7.452082  14.683066  11.172011
  6.9589047 11.860306   7.5576696  9.166423   9.940269  13.413978
  7.604774   9.881005   8.005665  13.008743  11.017416  13.731189
  8.67215

#### LGBM 개별 모델

In [ ]:
# lgb
lgb_reg=LGBMRegressor(n_estimators=100,
                      colsample_bytree=0.7,
                      learning_rate=0.15,
                      max_depth=3,
                      num_leaves=32,
                      subsample=0.6,
                      reg_alpha=0.01,
                      reg_lambda=0.1)
lgb_reg.fit(X_train,y_train)
lgb_pred=lgb_reg.predict(X_test)
print(lgb_pred)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 340
[LightGBM] [Info] Number of data points in the train set: 480, number of used features: 36
[LightGBM] [Info] Start training from score 10.088930
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

### 혼합 모델 예측

In [ ]:
# 개별 모델 예측값 혼합으로 최종 예측값 도출
pred=0.7*xgb_pred+0.3*lgb_pred
preds={'최종 혼합':pred,
       'xgb':xgb_pred,
       'lgb':lgb_pred}

# 최종 혼합 모델, 개별 모델의 RMSE 값 출력
get_rmse_pred(preds)

최종 혼합 모델의 RMSLE: 2.042
xgb 모델의 RMSLE: 2.041
lgb 모델의 RMSLE: 2.096


### 이후 아이디어

XGBoost 모델로 제출 시 리더보드 점수: 1480491.62514

앙상블 학습 -> **소프트보팅** 적용

## Dacon 리더보드 제출 파일 생성 (초기화)

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

# 1. 원본 테스트 데이터 로드 (movie_id를 얻기 위함)
original_test_df = pd.read_csv('/content/drive/MyDrive/Colab/ESAA/movies_test.csv')

# 2. 테스트 데이터 전처리 (train 데이터와 동일하게)
test_processed = original_test_df.copy()

# 결측치 처리
test_processed['dir_prev_bfnum'].fillna(0, inplace=True)

# 날짜 처리
test_processed['release_time'] = pd.to_datetime(test_processed['release_time'])
test_processed['year'] = test_processed['release_time'].dt.year
test_processed['month'] = test_processed['release_time'].dt.month
test_processed['weekday'] = test_processed['release_time'].dt.day_of_week

# 로그 변환
cols_to_transform_test = ['dir_prev_bfnum', 'num_staff', 'num_actor']
for col in cols_to_transform_test:
    test_processed[col + "_log"] = np.log1p(test_processed[col])

# 제거할 원본 변수들
cols_to_drop_test = [
    'title', 'director', 'release_time',
    'distributor',
    'num_actor', 'num_staff', 'dir_prev_bfnum',
]
test_processed = test_processed.drop(columns=cols_to_drop_test)
test_processed = test_processed.drop(columns=['dir_prev_num'])

# One-Hot Encoding
cols_to_ohe_test = ['genre', 'screening_rat', 'year', 'month', 'weekday']
test_processed = pd.get_dummies(test_processed, columns=cols_to_ohe_test)

# 신인 감독 여부 파생 변수
test_processed['is_rookie_director'] = (test_processed['dir_prev_bfnum_log'] == 0).astype(int)

# 3. 훈련 데이터와 테스트 데이터 컬럼 일치시키기
# X_features는 전처리된 훈련 데이터의 피처들입니다.
# test_processed의 컬럼을 X_features의 컬럼에 맞춤
test_aligned = test_processed.reindex(columns=X_features.columns, fill_value=0)

# 4. 최종 XGBoost 모델 재설정 및 학습 (가장 좋은 파라미터 사용)
final_xgb_model = XGBRegressor(
    colsample_bytree=0.5,
    learning_rate=0.05,
    max_depth=3,
    n_estimators=100,
    subsample=0.5,
    reg_alpha=0.01,
    reg_lambda=1,
    random_state=0,
    n_jobs=-1
)

# 전체 훈련 데이터로 모델 학습
final_xgb_model.fit(X_features, y_target)

# 5. 예측 수행
predictions_log = final_xgb_model.predict(test_aligned)

# 6. 로그 역변환 (expm1)
predictions = np.expm1(predictions_log)

# 7. submission 파일 생성
# 'title'이 왼쪽, 'box_off_num'이 오른쪽으로만 구성
submission = pd.DataFrame({'title': original_test_df['title'], 'box_off_num': predictions})

# 8. submission.csv 파일로 저장
submission.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' has been created.")
print(submission.head())

Submission file 'submission.csv' has been created.
         title   box_off_num
0       용서는 없다  1.053939e+06
1  아빠가 여자를 좋아해  4.493275e+05
2          하모니  7.000471e+05
3          의형제  1.069154e+06
4        평행 이론  7.321439e+05
